### Enter Fred API key here to run notebook:

In [ ]:
fred_api = ************

In [1]:
import requests
import numpy as np
import pandas as pd
import json

from dateutil.parser import parse

import datetime

import yfinance as yf
from fredapi import Fred 

from bs4 import BeautifulSoup
import re

import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier

fred = Fred(api_key=f'{fred_api}')

## Yahoo Finance example

In [2]:
SP500 = yf.Ticker("^GSPC")
SP500_history = SP500.history(period='max')
SP500_history

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1927-12-30 00:00:00-05:00,17.660000,17.660000,17.660000,17.660000,0,0.0,0.0
1928-01-03 00:00:00-05:00,17.760000,17.760000,17.760000,17.760000,0,0.0,0.0
1928-01-04 00:00:00-05:00,17.719999,17.719999,17.719999,17.719999,0,0.0,0.0
1928-01-05 00:00:00-05:00,17.549999,17.549999,17.549999,17.549999,0,0.0,0.0
1928-01-06 00:00:00-05:00,17.660000,17.660000,17.660000,17.660000,0,0.0,0.0
...,...,...,...,...,...,...,...
2024-04-29 00:00:00-04:00,5114.129883,5123.490234,5088.649902,5116.169922,3447450000,0.0,0.0
2024-04-30 00:00:00-04:00,5103.779785,5110.830078,5035.310059,5035.689941,4082470000,0.0,0.0
2024-05-01 00:00:00-04:00,5029.029785,5096.120117,5013.450195,5018.390137,4544170000,0.0,0.0


In [3]:
SP500_history = SP500_history.reset_index(drop=False)
SP500_history.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24201 entries, 0 to 24200
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype                           
---  ------        --------------  -----                           
 0   Date          24201 non-null  datetime64[ns, America/New_York]
 1   Open          24201 non-null  float64                         
 2   High          24201 non-null  float64                         
 3   Low           24201 non-null  float64                         
 4   Close         24201 non-null  float64                         
 5   Volume        24201 non-null  int64                           
 6   Dividends     24201 non-null  float64                         
 7   Stock Splits  24201 non-null  float64                         
dtypes: datetime64[ns, America/New_York](1), float64(6), int64(1)
memory usage: 1.5 MB


In [4]:
SP500_history['Date'] = SP500_history['Date'].dt.date
SP500_history = SP500_history.set_index('Date')
    
date_range = pd.date_range(start=SP500_history.index.min(), end=SP500_history.index.max(), freq='D')
SP500_history = SP500_history.reindex(date_range).ffill()
    
SP500_history = SP500_history.reset_index(drop=False)
SP500_history.rename(columns={'index':'model_date'},inplace=True)

In [5]:
SP500_history

,model_date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,1927-12-30,17.660000,17.660000,17.660000,17.660000,0.000000e+00,0.0,0.0
1,1927-12-31,17.660000,17.660000,17.660000,17.660000,0.000000e+00,0.0,0.0
2,1928-01-01,17.660000,17.660000,17.660000,17.660000,0.000000e+00,0.0,0.0
3,1928-01-02,17.660000,17.660000,17.660000,17.660000,0.000000e+00,0.0,0.0
4,1928-01-03,17.760000,17.760000,17.760000,17.760000,0.000000e+00,0.0,0.0
...,...,...,...,...,...,...,...,...
35185,2024-04-29,5114.129883,5123.490234,5088.649902,5116.169922,3.447450e+09,0.0,0.0
35186,2024-04-30,5103.779785,5110.830078,5035.310059,5035.689941,4.082470e+09,0.0,0.0
35187,2024-05-01,5029.029785,5096.120117,5013.450195,5018.390137,4.544170e+09,0.0,0.0
35188,2024-05-02,5049.319824,5073.209961,5011.049805,5064.200195,4.381660e+09,0.0,0.0


## For-loop for multiple tickers from Yahoo Finance

In [6]:
yf_series = ['^GSPC','GC=F','DX=F','JPY=X','^VIX']

In [7]:
yf_dfs=[] # empty list to store separate each series' dataframe

for series in yf_series:
    ticker = yf.Ticker(series)
    df = ticker.history(period='max') # get entire history of ticker
    
    df = df.reset_index(drop=False)
    df['Date'] = df['Date'].dt.date # I want to remove the time component from date
    
    df = df.set_index('Date') 
    
    # In order to avoid business holidays, weekends, and days when values from different data sources do not align,
        # I am creating a date series of every calendar day.
        # For days that there is no new value, it will take the last known value (using forward fill).
    date_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='D')
    df = df.reindex(date_range).ffill()
    
    df = df.reset_index(drop=False)
    
    # I am interested in Close price only, so this will be my 'value' for the series
    df.rename(columns={'index':'model_date','Close':'value'},inplace=True)
    
    df['series_id'] = series # create a column of series name to work with pivot table later on
    
    df = df.drop(columns=['Open','High','Low','Volume','Dividends','Stock Splits'])
    
    yf_dfs.append(df) # append each dataframe to list

In [8]:
yf_dfs

[      model_date        value series_id
 0     1927-12-30    17.660000     ^GSPC
 1     1927-12-31    17.660000     ^GSPC
 2     1928-01-01    17.660000     ^GSPC
 3     1928-01-02    17.660000     ^GSPC
 4     1928-01-03    17.760000     ^GSPC
 ...          ...          ...       ...
 35185 2024-04-29  5116.169922     ^GSPC
 35186 2024-04-30  5035.689941     ^GSPC
 35187 2024-05-01  5018.390137     ^GSPC
 35188 2024-05-02  5064.200195     ^GSPC
 35189 2024-05-03  5127.790039     ^GSPC
 
 [35190 rows x 3 columns],
      model_date        value series_id
 0    2000-08-30   273.899994      GC=F
 1    2000-08-31   278.299988      GC=F
 2    2000-09-01   277.000000      GC=F
 3    2000-09-02   277.000000      GC=F
 4    2000-09-03   277.000000      GC=F
 ...         ...          ...       ...
 8643 2024-04-29  2345.399902      GC=F
 8644 2024-04-30  2291.399902      GC=F
 8645 2024-05-01  2299.899902      GC=F
 8646 2024-05-02  2299.199951      GC=F
 8647 2024-05-03  2310.100098      GC=F

## Office of Financial Research dataframes:

In [9]:
#https://medium.com/analytics-vidhya/an-easy-technique-for-web-scraping-an-interactive-web-chart-38f5f945ca63

url = 'https://www.financialresearch.gov/financial-stress-index/data/fsi.json'
OFR_df = pd.read_json(url)
OFR_df

,OFRFSI,Credit,Equity_Valuation,Flight_to_Safety,Funding,Volatility,US,AE,EM
name,FSI,Credit,Equity valuation,Safe assets,Funding,Volatility,United States,Other advanced economies,Emerging markets
data,"[[946857600000, 2.14], [946944000000, 2.421], ...","[[946857600000, 0.54], [946944000000, 0.604], ...","[[946857600000, -0.051000000000000004], [94694...","[[946857600000, 0.67], [946944000000, 0.627], ...","[[946857600000, 0.47200000000000003], [9469440...","[[946857600000, 0.509], [946944000000, 0.561],...","[[946857600000, 1.7690000000000001], [94694400...","[[946857600000, 0.521], [946944000000, 0.47400...","[[946857600000, -0.15], [946944000000, -0.137]..."


In [10]:
expanded_rows = [] # empty list to store rows

for col in OFR_df.columns:
    for date_value in OFR_df[col][1]: # index 1 because data is in second row of each column
        expanded_rows.append({        # create dictionary for each observation and append to list
            'series_id': col,
            'date': date_value[0],    # date component in each list
            'value': date_value[1]    # value component in each list
        })

OFR_df2 = pd.DataFrame(expanded_rows) # convert list of dictionaries to DataFrame
OFR_df2

,series_id,date,value
0,OFRFSI,946857600000,2.140
1,OFRFSI,946944000000,2.421
2,OFRFSI,947030400000,2.297
3,OFRFSI,947116800000,2.292
4,OFRFSI,947203200000,2.005
...,...,...,...
55255,EM,1714003200000,-0.200
55256,EM,1714089600000,-0.202
55257,EM,1714348800000,-0.200
55258,EM,1714435200000,-0.198


In [11]:
OFR_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55260 entries, 0 to 55259
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   series_id  55260 non-null  object 
 1   date       55260 non-null  int64  
 2   value      55260 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.3+ MB


In [12]:
# Pivot table to create column for each series id
OFR_pivot = OFR_df2.pivot(index='date', columns='series_id', values='value')

OFR_pivot = OFR_pivot.reset_index()

OFR_pivot

series_id,date,AE,Credit,EM,Equity_Valuation,Flight_to_Safety,Funding,OFRFSI,US,Volatility
0,946857600000,0.521,0.540,-0.150,-0.051,0.670,0.472,2.140,1.769,0.509
1,946944000000,0.474,0.604,-0.137,0.079,0.627,0.550,2.421,2.084,0.561
2,947030400000,0.406,0.575,-0.132,0.080,0.653,0.501,2.297,2.023,0.488
3,947116800000,0.355,0.546,-0.127,0.082,0.581,0.566,2.292,2.064,0.517
4,947203200000,0.367,0.662,-0.131,-0.011,0.519,0.565,2.005,1.769,0.270
...,...,...,...,...,...,...,...,...,...,...
6135,1714003200000,-0.434,-0.717,-0.200,-0.092,-0.255,-0.174,-1.631,-0.997,-0.392
6136,1714089600000,-0.486,-0.744,-0.202,-0.105,-0.268,-0.170,-1.695,-1.006,-0.408
6137,1714348800000,-0.483,-0.734,-0.200,-0.109,-0.262,-0.187,-1.745,-1.061,-0.453
6138,1714435200000,-0.483,-0.725,-0.198,-0.100,-0.270,-0.188,-1.671,-0.991,-0.388


In [13]:
# Convert date column to a datetime object instead of int64
# x / 1000 is to convert milliseconds to seconds
OFR_pivot['date'] = OFR_pivot['date'].apply(lambda x: datetime.datetime.fromtimestamp(x / 1000))
OFR_pivot

series_id,date,AE,Credit,EM,Equity_Valuation,Flight_to_Safety,Funding,OFRFSI,US,Volatility
0,2000-01-02 19:00:00,0.521,0.540,-0.150,-0.051,0.670,0.472,2.140,1.769,0.509
1,2000-01-03 19:00:00,0.474,0.604,-0.137,0.079,0.627,0.550,2.421,2.084,0.561
2,2000-01-04 19:00:00,0.406,0.575,-0.132,0.080,0.653,0.501,2.297,2.023,0.488
3,2000-01-05 19:00:00,0.355,0.546,-0.127,0.082,0.581,0.566,2.292,2.064,0.517
4,2000-01-06 19:00:00,0.367,0.662,-0.131,-0.011,0.519,0.565,2.005,1.769,0.270
...,...,...,...,...,...,...,...,...,...,...
6135,2024-04-24 20:00:00,-0.434,-0.717,-0.200,-0.092,-0.255,-0.174,-1.631,-0.997,-0.392
6136,2024-04-25 20:00:00,-0.486,-0.744,-0.202,-0.105,-0.268,-0.170,-1.695,-1.006,-0.408
6137,2024-04-28 20:00:00,-0.483,-0.734,-0.200,-0.109,-0.262,-0.187,-1.745,-1.061,-0.453
6138,2024-04-29 20:00:00,-0.483,-0.725,-0.198,-0.100,-0.270,-0.188,-1.671,-0.991,-0.388


In [14]:
# Dates are marked as one day prior to observation day
# so add one day to align them
OFR_pivot['date'] = OFR_pivot['date'] + datetime.timedelta(days=1)


OFR_pivot['date']=OFR_pivot['date'].dt.date # remove time component

OFR_pivot

series_id,date,AE,Credit,EM,Equity_Valuation,Flight_to_Safety,Funding,OFRFSI,US,Volatility
0,2000-01-03,0.521,0.540,-0.150,-0.051,0.670,0.472,2.140,1.769,0.509
1,2000-01-04,0.474,0.604,-0.137,0.079,0.627,0.550,2.421,2.084,0.561
2,2000-01-05,0.406,0.575,-0.132,0.080,0.653,0.501,2.297,2.023,0.488
3,2000-01-06,0.355,0.546,-0.127,0.082,0.581,0.566,2.292,2.064,0.517
4,2000-01-07,0.367,0.662,-0.131,-0.011,0.519,0.565,2.005,1.769,0.270
...,...,...,...,...,...,...,...,...,...,...
6135,2024-04-25,-0.434,-0.717,-0.200,-0.092,-0.255,-0.174,-1.631,-0.997,-0.392
6136,2024-04-26,-0.486,-0.744,-0.202,-0.105,-0.268,-0.170,-1.695,-1.006,-0.408
6137,2024-04-29,-0.483,-0.734,-0.200,-0.109,-0.262,-0.187,-1.745,-1.061,-0.453
6138,2024-04-30,-0.483,-0.725,-0.198,-0.100,-0.270,-0.188,-1.671,-0.991,-0.388


In [15]:
# Values are published two days after their corresponding observation date.
# Therefore, model_date represents the date a value becomes known.
# Model_date is the date used when modelling in order to avoid any forward-looking bias

OFR_pivot['model_date'] = pd.to_datetime(OFR_pivot['date'])+ datetime.timedelta(days=2)
OFR_pivot.drop(columns=['date'],inplace=True)
OFR_pivot

series_id,AE,Credit,EM,Equity_Valuation,Flight_to_Safety,Funding,OFRFSI,US,Volatility,model_date
0,0.521,0.540,-0.150,-0.051,0.670,0.472,2.140,1.769,0.509,2000-01-05
1,0.474,0.604,-0.137,0.079,0.627,0.550,2.421,2.084,0.561,2000-01-06
2,0.406,0.575,-0.132,0.080,0.653,0.501,2.297,2.023,0.488,2000-01-07
3,0.355,0.546,-0.127,0.082,0.581,0.566,2.292,2.064,0.517,2000-01-08
4,0.367,0.662,-0.131,-0.011,0.519,0.565,2.005,1.769,0.270,2000-01-09
...,...,...,...,...,...,...,...,...,...,...
6135,-0.434,-0.717,-0.200,-0.092,-0.255,-0.174,-1.631,-0.997,-0.392,2024-04-27
6136,-0.486,-0.744,-0.202,-0.105,-0.268,-0.170,-1.695,-1.006,-0.408,2024-04-28
6137,-0.483,-0.734,-0.200,-0.109,-0.262,-0.187,-1.745,-1.061,-0.453,2024-05-01
6138,-0.483,-0.725,-0.198,-0.100,-0.270,-0.188,-1.671,-0.991,-0.388,2024-05-02


In [16]:
OFR_pivot = OFR_pivot.set_index('model_date')


# In order to avoid business holidays, weekends, and days when values from different data sources do not align,
    # I am creating a date series of every calendar day.
    # For days that there is no new value, it will take the last known value (using forward fill).
date_range = pd.date_range(start=OFR_pivot.index.min(), end=OFR_pivot.index.max(), freq='D')
OFR_pivot = OFR_pivot.reindex(date_range).ffill()
    
OFR_pivot = OFR_pivot.reset_index(drop=False)
OFR_pivot.rename(columns={'index':'model_date'},inplace=True)
OFR_pivot

series_id,model_date,AE,Credit,EM,Equity_Valuation,Flight_to_Safety,Funding,OFRFSI,US,Volatility
0,2000-01-05,0.521,0.540,-0.150,-0.051,0.670,0.472,2.140,1.769,0.509
1,2000-01-06,0.474,0.604,-0.137,0.079,0.627,0.550,2.421,2.084,0.561
2,2000-01-07,0.406,0.575,-0.132,0.080,0.653,0.501,2.297,2.023,0.488
3,2000-01-08,0.355,0.546,-0.127,0.082,0.581,0.566,2.292,2.064,0.517
4,2000-01-09,0.367,0.662,-0.131,-0.011,0.519,0.565,2.005,1.769,0.270
...,...,...,...,...,...,...,...,...,...,...
8881,2024-04-29,-0.486,-0.744,-0.202,-0.105,-0.268,-0.170,-1.695,-1.006,-0.408
8882,2024-04-30,-0.486,-0.744,-0.202,-0.105,-0.268,-0.170,-1.695,-1.006,-0.408
8883,2024-05-01,-0.483,-0.734,-0.200,-0.109,-0.262,-0.187,-1.745,-1.061,-0.453
8884,2024-05-02,-0.483,-0.725,-0.198,-0.100,-0.270,-0.188,-1.671,-0.991,-0.388


In [17]:
OFR_features=[] # empty list to store separate dataframe for each series

# for-loop to create a dataframe for each series
for column in OFR_pivot.columns:
    if column != 'model_date':
        df = pd.DataFrame({
            'model_date': OFR_pivot['model_date'],
            'value': OFR_pivot[column],
            'series_id': column
        })
        OFR_features.append(df)
OFR_features

[     model_date  value series_id
 0    2000-01-05  0.521        AE
 1    2000-01-06  0.474        AE
 2    2000-01-07  0.406        AE
 3    2000-01-08  0.355        AE
 4    2000-01-09  0.367        AE
 ...         ...    ...       ...
 8881 2024-04-29 -0.486        AE
 8882 2024-04-30 -0.486        AE
 8883 2024-05-01 -0.483        AE
 8884 2024-05-02 -0.483        AE
 8885 2024-05-03 -0.473        AE
 
 [8886 rows x 3 columns],
      model_date  value series_id
 0    2000-01-05  0.540    Credit
 1    2000-01-06  0.604    Credit
 2    2000-01-07  0.575    Credit
 3    2000-01-08  0.546    Credit
 4    2000-01-09  0.662    Credit
 ...         ...    ...       ...
 8881 2024-04-29 -0.744    Credit
 8882 2024-04-30 -0.744    Credit
 8883 2024-05-01 -0.734    Credit
 8884 2024-05-02 -0.725    Credit
 8885 2024-05-03 -0.718    Credit
 
 [8886 rows x 3 columns],
      model_date  value series_id
 0    2000-01-05 -0.150        EM
 1    2000-01-06 -0.137        EM
 2    2000-01-07 -0.132   

## FRED Database dataframes:

### Using FRED API:

In [18]:
url = f'https://api.stlouisfed.org/fred/series/observations?series_id=DGS10&api_key={fred_api}&file_type=json'
response = requests.get(url)
data = response.json()
data

{'realtime_start': '2024-05-03',
 'realtime_end': '2024-05-03',
 'observation_start': '1600-01-01',
 'observation_end': '9999-12-31',
 'units': 'lin',
 'output_type': 1,
 'file_type': 'json',
 'order_by': 'observation_date',
 'sort_order': 'asc',
 'count': 16263,
 'offset': 0,
 'limit': 100000,
 'observations': [{'realtime_start': '2024-05-03',
   'realtime_end': '2024-05-03',
   'date': '1962-01-02',
   'value': '4.06'},
  {'realtime_start': '2024-05-03',
   'realtime_end': '2024-05-03',
   'date': '1962-01-03',
   'value': '4.03'},
  {'realtime_start': '2024-05-03',
   'realtime_end': '2024-05-03',
   'date': '1962-01-04',
   'value': '3.99'},
  {'realtime_start': '2024-05-03',
   'realtime_end': '2024-05-03',
   'date': '1962-01-05',
   'value': '4.02'},
  {'realtime_start': '2024-05-03',
   'realtime_end': '2024-05-03',
   'date': '1962-01-08',
   'value': '4.03'},
  {'realtime_start': '2024-05-03',
   'realtime_end': '2024-05-03',
   'date': '1962-01-09',
   'value': '4.05'},
  {'

'Data' is a dictionary. The observations are located as the value of the 'observations' key, as a list of dictionaries for each observation.

In [19]:
# Dataframe creation:
DGS10_df = pd.DataFrame(data['observations'])
DGS10_df = DGS10_df[['date', 'value']] 
DGS10_df['series_id'] = 'DGS10'
DGS10_df.rename(columns={'date':'model_date'},inplace=True)
DGS10_df['model_date'] = pd.to_datetime(DGS10_df['model_date'])
DGS10_df['model_date'] = DGS10_df['model_date'].dt.date
DGS10_df['model_date'] = pd.to_datetime(DGS10_df['model_date'])
DGS10_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16263 entries, 0 to 16262
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   model_date  16263 non-null  datetime64[ns]
 1   value       16263 non-null  object        
 2   series_id   16263 non-null  object        
dtypes: datetime64[ns](1), object(2)
memory usage: 381.3+ KB


In [20]:
daily_series = ['DGS10','DGS2','DGS5','DGS30','DGS20','DGS3MO','DGS1MO','DGS3','DGS6MO','DGS7',
               'DFF','RRPONTSYD','BAMLH0A0HYM2','T10YIE','DFII10','T5YIE','DAAA','DTB3',
                'BAMLH0A0HYM2EY','BAMLC0A0CM','BAMLHE00EHYIOAS','BAMLEMRACRPIASIAOAS','BAMLEMCBPIOAS']

In [43]:
daily_dfs = [] # empty list to store the dataframes for each series

for series_id in daily_series:
    url = f'https://api.stlouisfed.org/fred/series/observations?series_id={series_id}&api_key={fred_api}&file_type=json'
    response = requests.get(url)
    data = response.json()
    
    # Dataframe creation:
    df = pd.DataFrame(data['observations'])
    df = df[['date', 'value']] # retrieve the date and value
    df['series_id'] = series_id # create a column that has series id for use in pivot table later
    
    # Observations for daily series can be found on the same day for which the observation is associated.
    # Therefore, the observation date and model date will be the same
    df.rename(columns={'date':'model_date'},inplace=True)
    
    df['model_date'] = pd.to_datetime(df['model_date']) # convert to datetime type
    df['model_date'] = df['model_date'].dt.date         # make sure there is no time component
    df['model_date'] = pd.to_datetime(df['model_date']) # convert back to datetime type
    
    daily_dfs.append(df) # append each dataframe to the list
    
daily_dfs[0]

,model_date,value,series_id
0,1962-01-02,4.06,DGS10
1,1962-01-03,4.03,DGS10
2,1962-01-04,3.99,DGS10
3,1962-01-05,4.02,DGS10
4,1962-01-08,4.03,DGS10
...,...,...,...
16258,2024-04-26,4.67,DGS10
16259,2024-04-29,4.63,DGS10
16260,2024-04-30,4.69,DGS10
16261,2024-05-01,4.63,DGS10


### Using FRED Python package to get realtime dates

### M2SL example:

In [22]:
M2SL_df = fred.get_series_all_releases('M2SL')
    
M2SL_df.rename(columns={'date':'obs_date'},inplace=True)
M2SL_df['realtime_start'] = M2SL_df['realtime_start'].dt.date
M2SL_df['obs_date'] = M2SL_df['obs_date'].dt.date
M2SL_df['date_lag'] = M2SL_df['realtime_start']-M2SL_df['obs_date']
M2SL_df

,realtime_start,obs_date,value,date_lag
0,1980-02-08,1959-01-01,285.5,7708 days
1,1981-06-26,1959-01-01,286.0,8212 days
2,1982-02-05,1959-01-01,286.5,8436 days
3,1984-02-16,1959-01-01,286.6,9177 days
4,1988-02-18,1959-01-01,286.7,10640 days
...,...,...,...,...
26331,2024-03-26,2024-01-01,20790.2,85 days
26332,2024-04-23,2024-01-01,20754.2,113 days
26333,2024-03-26,2024-02-01,20783.6,54 days
26334,2024-04-23,2024-02-01,20748.6,82 days


In [23]:
weekly_series = ['MORTGAGE30US','MORTGAGE15US','WALCL','ICSA']

monthly_series = ['M2SL','CPIAUCSL','UNRATE','CSUSHPINSA','M1SL','CORESTICKM159SFRBATL',
                  'PCE','CIVPART','PSAVERT','PAYEMS','CPILFESL']

quarterly_series = ['GDP','GDPC1','MSPUS','DRCCLACBS','ASPUS','GFDEGDQ188S','GFDEBTN','M2V']

In [24]:
def FRED_series_WMQ(dict1, dict2, series, max_lag_periods):    
    
    df = fred.get_series_all_releases(series)
    
    df.rename(columns={'date':'obs_date'},inplace=True)
    df['realtime_start'] = df['realtime_start'].dt.date
    df['obs_date'] = df['obs_date'].dt.date
    df['date_lag'] = df['realtime_start']-df['obs_date']  # find lag between observation date and publish date
    
    
    # keep only the first time observation was published
    df_first = df.drop_duplicates(subset='obs_date',keep='first').copy()
    
    # find the max lag between publish date and observation date over the last max_lag_periods
    max_lag = df_first['date_lag'].iloc[-max_lag_periods:].max()
    
    
    # apply max lag to observation date only if publish (realtime) date is longer than max lag:
    
    df['model_date'] = df.apply(lambda x: x['obs_date']+max_lag if x['date_lag']>max_lag
                               else x['realtime_start'],axis=1)
    
    df_first['model_date'] = df.apply(lambda x: x['obs_date']+max_lag if x['date_lag']>max_lag
                               else x['realtime_start'],axis=1)
    
    df['model_date'] = pd.to_datetime(df['model_date'])
    df_first['model_date'] = pd.to_datetime(df['model_date'])
    
    df_first = df_first.set_index('model_date')
    
    
    # In order to avoid business holidays, weekends, and days when values from different data sources do not align,
        # I am creating a date series of every calendar day.
        # For days that there is no new value, it will take the last known value (using forward fill).
    date_range = pd.date_range(start=df_first.index.min(), end=df_first.index.max(), freq='D')
    df_first = df_first.reindex(date_range).ffill()
    
    df_first = df_first.reset_index(drop=False)
    df_first.rename(columns={'index':'model_date'},inplace=True)
    
    df_first['series_id'] = series
    
    df_first = df_first.drop(columns=['realtime_start','obs_date','date_lag'])
    
    dict1[series] = df
    dict2[series] = df_first

In [25]:
weekly_rt_dict = {}     # empty dictionary to store each dataframe
weekly_rt_first_dict={} # empty dictionary to store each dataframe with duplicate observations removed

for series in weekly_series:
    FRED_series_WMQ(weekly_rt_dict,weekly_rt_first_dict,series,260)

In [26]:
monthly_rt_dict = {}     # empty dictionary to store each dataframe
monthly_rt_first_dict={} # empty dictionary to store each dataframe with duplicate observations removed

for series in monthly_series:
    FRED_series_WMQ(monthly_rt_dict,monthly_rt_first_dict,series,60)

In [27]:
quarterly_rt_dict = {}     # empty dictionary to store each dataframe
quarterly_rt_first_dict={} # empty dictionary to store each dataframe with duplicate observations removed

for series in quarterly_series:
    FRED_series_WMQ(quarterly_rt_dict,quarterly_rt_first_dict,series,20)

In [28]:
weekly_rt_first_dict['ICSA']

,model_date,value,series_id
0,1967-01-12,208000.0,ICSA
1,1967-01-13,208000.0,ICSA
2,1967-01-14,208000.0,ICSA
3,1967-01-15,208000.0,ICSA
4,1967-01-16,208000.0,ICSA
...,...,...,...
20926,2024-04-28,207000.0,ICSA
20927,2024-04-29,207000.0,ICSA
20928,2024-04-30,207000.0,ICSA
20929,2024-05-01,207000.0,ICSA


In [29]:
monthly_rt_first_dict['M2SL']

,model_date,value,series_id
0,1959-02-28,285.5,M2SL
1,1959-03-01,285.5,M2SL
2,1959-03-02,285.5,M2SL
3,1959-03-03,285.5,M2SL
4,1959-03-04,285.5,M2SL
...,...,...,...
23792,2024-04-19,20783.6,M2SL
23793,2024-04-20,20783.6,M2SL
23794,2024-04-21,20783.6,M2SL
23795,2024-04-22,20783.6,M2SL


In [30]:
quarterly_rt_first_dict['GDP']

,model_date,value,series_id
0,1946-05-02,199.700,GDP
1,1946-05-03,199.700,GDP
2,1946-05-04,199.700,GDP
3,1946-05-05,199.700,GDP
4,1946-05-06,199.700,GDP
...,...,...,...
28479,2024-04-21,27938.831,GDP
28480,2024-04-22,27938.831,GDP
28481,2024-04-23,27938.831,GDP
28482,2024-04-24,27938.831,GDP


In [31]:
all_dfs = [] # empty list to store all dataframes

# store weekly, monthly, and quarterly FRED series in list:
for d in [weekly_rt_first_dict,monthly_rt_first_dict,quarterly_rt_first_dict]:
    for df in d.values():
        all_dfs.append(df)

In [32]:
all_dfs.extend(daily_dfs) # extend to include daily FRED series
all_dfs.extend(OFR_features) # extend to include OFR series
all_dfs.extend(yf_dfs) # extend to include yfinance series
all_dfs

[      model_date  value     series_id
 0     1971-04-04   7.33  MORTGAGE30US
 1     1971-04-05   7.33  MORTGAGE30US
 2     1971-04-06   7.33  MORTGAGE30US
 3     1971-04-07   7.33  MORTGAGE30US
 4     1971-04-08   7.33  MORTGAGE30US
 ...          ...    ...           ...
 19383 2024-04-28   7.17  MORTGAGE30US
 19384 2024-04-29   7.17  MORTGAGE30US
 19385 2024-04-30   7.17  MORTGAGE30US
 19386 2024-05-01   7.17  MORTGAGE30US
 19387 2024-05-02   7.22  MORTGAGE30US
 
 [19388 rows x 3 columns],
       model_date  value     series_id
 0     1991-09-01   8.77  MORTGAGE15US
 1     1991-09-02   8.77  MORTGAGE15US
 2     1991-09-03   8.77  MORTGAGE15US
 3     1991-09-04   8.77  MORTGAGE15US
 4     1991-09-05   8.77  MORTGAGE15US
 ...          ...    ...           ...
 11928 2024-04-28   6.44  MORTGAGE15US
 11929 2024-04-29   6.44  MORTGAGE15US
 11930 2024-04-30   6.44  MORTGAGE15US
 11931 2024-05-01   6.44  MORTGAGE15US
 11932 2024-05-02   6.47  MORTGAGE15US
 
 [11933 rows x 3 columns],
      

In [59]:
all_data = pd.concat(all_dfs, ignore_index=True) # concatenate all dfs from all_dfs list

all_data = all_data.pivot(index='model_date', columns='series_id', values='value') # Pivot to structure by model_date and series_id

all_data

series_id,AE,ASPUS,BAMLC0A0CM,BAMLEMCBPIOAS,BAMLEMRACRPIASIAOAS,BAMLH0A0HYM2,BAMLH0A0HYM2EY,BAMLHE00EHYIOAS,CIVPART,CORESTICKM159SFRBATL,...,PSAVERT,RRPONTSYD,T10YIE,T5YIE,UNRATE,US,Volatility,WALCL,^GSPC,^VIX
model_date,,,,,,,,,,,,,,,,,,,,,
1927-12-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.66,NaN
1927-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.66,NaN
1928-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.66,NaN
1928-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.66,NaN
1928-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.76,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-30,-0.486,NaN,0.91,1.90,1.29,3.18,7.99,3.50,62.7,NaN,...,NaN,534.234,2.41,2.43,3.8,-1.006,-0.408,7402434.0,5035.689941,15.65
2024-05-01,-0.483,NaN,0.91,1.94,1.30,3.21,7.95,3.50,62.7,NaN,...,NaN,438.148,2.38,2.39,3.8,-1.061,-0.453,7402434.0,5018.390137,15.39
2024-05-02,-0.483,NaN,0.90,1.95,1.30,3.16,7.83,3.51,62.7,NaN,...,NaN,428.68,2.37,2.37,3.8,-0.991,-0.388,7362474.0,5064.200195,14.68


In [60]:
all_data = all_data.replace('.', pd.np.nan) # replace '.' with NaN
all_data = all_data.fillna(method='ffill') # replace NaN with last known value

# convert all value columns to float type
for col in all_data.columns:
    all_data[col] = all_data[col].astype(float)

all_data.info()

C:\Users\footb\AppData\Local\Temp\ipykernel_21388\2576946452.py:1: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  all_data = all_data.replace('.', pd.np.nan) # replace '.' with NaN


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 35191 entries, 1927-12-30 to 2024-05-04
Data columns (total 60 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   AE                    8887 non-null   float64
 1   ASPUS                 22286 non-null  float64
 2   BAMLC0A0CM            9987 non-null   float64
 3   BAMLEMCBPIOAS         9257 non-null   float64
 4   BAMLEMRACRPIASIAOAS   9257 non-null   float64
 5   BAMLH0A0HYM2          9987 non-null   float64
 6   BAMLH0A0HYM2EY        9987 non-null   float64
 7   BAMLHE00EHYIOAS       9622 non-null   float64
 8   CIVPART               27844 non-null  float64
 9   CORESTICKM159SFRBATL  20566 non-null  float64
 10  CPIAUCSL              28205 non-null  float64
 11  CPILFESL              24552 non-null  float64
 12  CSUSHPINSA            17931 non-null  float64
 13  Credit                8887 non-null   float64
 14  DAAA                  15098 non-null  float64
 15  DF

In [61]:
all_data.head()

series_id,AE,ASPUS,BAMLC0A0CM,BAMLEMCBPIOAS,BAMLEMRACRPIASIAOAS,BAMLH0A0HYM2,BAMLH0A0HYM2EY,BAMLHE00EHYIOAS,CIVPART,CORESTICKM159SFRBATL,...,PSAVERT,RRPONTSYD,T10YIE,T5YIE,UNRATE,US,Volatility,WALCL,^GSPC,^VIX
model_date,,,,,,,,,,,,,,,,,,,,,
1927-12-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.66,NaN
1927-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.66,NaN
1928-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.66,NaN
1928-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.66,NaN
1928-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.76,NaN


In [62]:
all_data.tail()

series_id,AE,ASPUS,BAMLC0A0CM,BAMLEMCBPIOAS,BAMLEMRACRPIASIAOAS,BAMLH0A0HYM2,BAMLH0A0HYM2EY,BAMLHE00EHYIOAS,CIVPART,CORESTICKM159SFRBATL,...,PSAVERT,RRPONTSYD,T10YIE,T5YIE,UNRATE,US,Volatility,WALCL,^GSPC,^VIX
model_date,,,,,,,,,,,,,,,,,,,,,
2024-04-30,-0.486,492300.0,0.91,1.90,1.29,3.18,7.99,3.50,62.7,4.509212,...,3.2,534.234,2.41,2.43,3.8,-1.006,-0.408,7402434.0,5035.689941,15.65
2024-05-01,-0.483,492300.0,0.91,1.94,1.30,3.21,7.95,3.50,62.7,4.509212,...,3.2,438.148,2.38,2.39,3.8,-1.061,-0.453,7402434.0,5018.390137,15.39
2024-05-02,-0.483,492300.0,0.90,1.95,1.30,3.16,7.83,3.51,62.7,4.509212,...,3.2,428.680,2.37,2.37,3.8,-0.991,-0.388,7362474.0,5064.200195,14.68
2024-05-03,-0.473,492300.0,0.90,1.95,1.30,3.16,7.83,3.51,62.7,4.509212,...,3.2,450.165,2.35,2.35,3.9,-0.994,-0.387,7362474.0,5127.790039,13.49
2024-05-04,-0.473,492300.0,0.90,1.95,1.30,3.16,7.83,3.51,62.7,4.509212,...,3.2,450.165,2.35,2.35,3.9,-0.994,-0.387,7362474.0,5127.790039,13.49


In [64]:
all_data.to_csv('all_data_max_daily.csv')